In [2]:
!pip install praw

In [30]:

import datetime

import praw
import pandas as pd

In [31]:
reddit = praw.Reddit( client_id='ICHtPaY7XyFzCA',
                      client_secret='gC7MBGLr-dOWMDJWtvxyGz0E4q4',
                      user_agent='android:my_app:v1 (by /u/Lord_vaider_1)')

In [53]:
tesla = reddit.subreddit('rickandmorty')

#Gathering the top 500 posts, with their title, url, body, upvotes, timestamp, and an index that serves as a key between the
#posts and the comments we collect later
posts = []
for index, post in enumerate(tesla.top(limit=300)):
    posts.append([post.title, "https://www.reddit.com" + post.permalink, post.selftext, post.score, post.created_utc, index])

#Converting into DataFrame
posts = pd.DataFrame(posts, columns=['Title', 'URL', 'Body', 'Upvotes', 'Time', 'Key'])
#Changing from utc time to standard timestamp
posts.Time = posts.Time.apply(lambda x: pd.to_datetime(datetime.datetime.fromtimestamp(x)))

#The first post is a sticky, so we can drop it
posts = posts.iloc[1:]

In [54]:
posts.head(3)

,Title,URL,Body,Upvotes,Time,Key
1,Are they real?,https://www.reddit.com/r/rickandmorty/comments...,,139497,2019-12-20 04:33:03,1
2,2020 has been wild.,https://www.reddit.com/r/rickandmorty/comments...,,86242,2020-09-07 23:27:33,2
3,Rick and Morty coming November 2019!!!,https://www.reddit.com/r/rickandmorty/comments...,,77201,2019-05-16 03:02:09,3


In [55]:
posts.shape

(299, 6)

In [56]:
def collect_replies(key, url):
    ''' 
    params pandas series row: each row of the dataframe we built above in the form of a panda series
    Returns a pandas DataFrame, where each row represents an individual comment
    '''
    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=None)
    comment_queue = submission.comments[:] 

    table = {'Reply':[], 'Upvote':[], 'Time':[], 'Key':[]}

    while comment_queue:
        comment = comment_queue.pop(0)
        table['Reply'].append(comment.body)
        table['Time'].append(comment.created_utc)
        table['Upvote'].append(comment.score)
        table['Key'].append(key)
        comment_queue.extend(comment.replies)
    
    return pd.DataFrame.from_dict(table)

In [57]:

#Let us first generate a list of tupules that contains the key and url for each row - the first value of the tupule is key,
#and the second value is url
keys = posts.Key.tolist()
urls = posts.URL.tolist()
tupules = list(zip(keys, urls))

#Now we generate our comments dataframe using list comprehensions!
comments = pd.concat([collect_replies(x[0], x[1]) for x in tupules])

In [58]:
#Again, converting the timestamp from utc to a standard format
comments.Time = comments.Time.apply(lambda x: pd.to_datetime(datetime.datetime.fromtimestamp(x)))

In [59]:
comments.head(3)

,Reply,Upvote,Time,Key
0,"For those out of the loop, people who have com...",4227,2019-12-20 05:54:14,1
1,I’m out!,192,2019-12-20 05:23:04,1
2,Wait we get coins for upvoting ?,205,2019-12-20 06:10:29,1


In [60]:

comments.shape

(157896, 4)

In [61]:
posts.shape

(299, 6)

In [62]:
posts.head(3)

,Title,URL,Body,Upvotes,Time,Key
1,Are they real?,https://www.reddit.com/r/rickandmorty/comments...,,139497,2019-12-20 04:33:03,1
2,2020 has been wild.,https://www.reddit.com/r/rickandmorty/comments...,,86242,2020-09-07 23:27:33,2
3,Rick and Morty coming November 2019!!!,https://www.reddit.com/r/rickandmorty/comments...,,77201,2019-05-16 03:02:09,3


In [64]:
comments.to_csv(r'C:\Users\Siddharth Tripathi\Desktop\DataScience_Pranjal\Kaggle_assessment\RNN_Sentiment\CommentsRick.csv', index=False)

In [65]:
posts.to_csv(r'C:\Users\Siddharth Tripathi\Desktop\DataScience_Pranjal\Kaggle_assessment\RNN_Sentiment\postsRick.csv', index=False)

As we can see, we generated two csv files using Reddit API

1) CommentsRick.csv
2) postsRick.csv

These files will be used for sentiment analysis using vader and textblob pre-trained models.
You can find the code in file named VaderBlob.ipynb. 

The code in this file has been taken from the example code given in MSA phase2 kit